In [3]:
import pandas as pd 
import os
import datetime as dt
phaiThuGocFile='D:/Tai/131111.xls'
phaiTraTaiFile='D:/Tai/331311.xls'
dfPhaiThuGoc=pd.read_excel(phaiThuGocFile,skiprows=7)
dfPhaiTraTai=pd.read_excel(phaiTraTaiFile,skiprows=7)

In [13]:
import datetime as dt
dfPhaiThuGoc=dfPhaiThuGoc[(dfPhaiThuGoc['Mã NT'].notna())|(dfPhaiThuGoc['Số chứng từ'].notna())]
dfPhaiThuGoc['DueDate']=pd.to_datetime(dfPhaiThuGoc['Hạn thanh toán '],dayfirst=True)
dfPhaiThuGoc['Ngay den han']=dfPhaiThuGoc.groupby(['Số đơn / EN '])['DueDate'].transform('max')
dfPhaiThuGoc2=dfPhaiThuGoc.groupby(['Số đơn / EN ','TênKH','Ngay den han'],as_index=False)['Dư cuối kỳ > VND'].sum()
dfPhaiThuGoc2['Ngay so sanh']=pd.to_datetime(dt.datetime.now().date(),dayfirst=True)
dfPhaiThuGoc2['Ngay qua han']=dfPhaiThuGoc2['Ngay den han']-dfPhaiThuGoc2['Ngay so sanh']
dfPhaiThuGoc2['Ngay qua han']=-dfPhaiThuGoc2['Ngay qua han'].dt.days.astype('float64')

In [76]:
dfPhaiTraTai=dfPhaiTraTai[(dfPhaiTraTai['Mã NT'].notna())|(dfPhaiTraTai['Số chứng từ'].notna())]
dfPhaiTraTai['DueDate']=pd.to_datetime(dfPhaiTraTai['Hạn thanh toán '],dayfirst=True)
dfPhaiTraTai['Ngay den han']=dfPhaiTraTai.groupby(['Số đơn / EN '])['DueDate'].transform('max')
dfPhaiTraTai2=dfPhaiTraTai.groupby(['Số đơn / EN ','Mã KH','Số kỳ','Account month','TênKH','Ngay den han'],as_index=False)['Dư cuối kỳ Nguyên tệ ','Dư cuối kỳ > VND'].sum()
dfPhaiTraTai2['Ngay so sanh']=pd.to_datetime(dt.datetime.now().date(),dayfirst=True)
dfPhaiTraTai2['Ngay qua han']=dfPhaiTraTai2['Ngay den han']-dfPhaiTraTai2['Ngay so sanh']
dfPhaiTraTai2['Ngay qua han']=-dfPhaiTraTai2['Ngay qua han'].dt.days.astype('float64')

In [77]:
import numpy as np
dfPhaiTraTai3=dfPhaiTraTai2.merge(dfPhaiThuGoc2,on='Số đơn / EN ',how='left',suffixes=(" Tái"," Gốc"))
dfPhaiTraTai3['Dư cuối kỳ > VND Gốc']=dfPhaiTraTai3['Dư cuối kỳ > VND Gốc'].fillna(0)
condition=dfPhaiTraTai3['Số đơn / EN '].str.contains('TTY|EXC|XOL')
dfPhaiTraTai3['FACT/TTY']=np.where(condition,'TTY','FACT')


In [78]:
dfPhaiTraTaiDuDieuKien=dfPhaiTraTai3.loc[(dfPhaiTraTai3['Ngay qua han Tái']>-30) & (dfPhaiTraTai3['Dư cuối kỳ > VND Gốc']==0)]

In [88]:
df131111_2016_2019=pd.read_hdf('D:/131111_2016_2019.h5')
df131111_2016_2019=df131111_2016_2019.loc[df131111_2016_2019['TK chi tiet_Bên có'].notna(),['Số chứng từ','Ngày chứng từ','Số đơn bảo hiểm','Số đơn chứng từ','TK chi tiet_Bên có','Số tiền_Bên có','Số tiền VND_Bên có']]




In [ ]:
df131111_2020=pd.read_excel('F:/DATA/131111_2016_2019/131111.XLS',sheet_name=None,header=None)

In [102]:
func=lambda x,y: x if len(str(x))>5 else y 

In [103]:
df131111_2016_2019['Policy']=df131111_2016_2019.apply(lambda x: func(x['Số đơn bảo hiểm'],x['Số đơn chứng từ']),axis=1)

In [106]:
df131111_2016_2019=df131111_2016_2019.loc[df131111_2016_2019['Policy'].isin(dfPhaiTraTaiDuDieuKien['Số đơn / EN '])]

In [107]:
df131111_2016_2019['PaidInfo']=df131111_2016_2019['Số chứng từ']+':'+df131111_2016_2019['Số chứng từ']+':'+df131111_2016_2019['Ngày chứng từ']

In [108]:
df131111_2016_2019.to_csv('D:/df131111_2016_2019.csv')

In [79]:
dfPhaiTraTaiChuaDuDieuKien=dfPhaiTraTai3.loc[(dfPhaiTraTai3['Ngay qua han Tái']>-30) & (dfPhaiTraTai3['Dư cuối kỳ > VND Gốc']!=0)]

In [80]:
writer = pd.ExcelWriter("D:/PhaiTraTai.xlsx",engine='xlsxwriter')
# Turn off the default header and skip one row to allow us to insert a
# user defined header.
dfPhaiTraTai3.to_excel(writer, sheet_name='Tong hop', startrow=1, header=False)
dfPhaiTraTaiDuDieuKien.to_excel(writer, sheet_name='CongNoTaiDuDieuKienTT', startrow=1, header=False)
dfPhaiTraTaiChuaDuDieuKien.to_excel(writer, sheet_name='CongNoTaiChuaDuDieuKienTT', startrow=1, header=False)
workbook  = writer.book
worksheet1 = writer.sheets['Tong hop']
worksheet2 = writer.sheets['CongNoTaiDuDieuKienTT']
worksheet3 = writer.sheets['CongNoTaiChuaDuDieuKienTT']


In [81]:
header_format = workbook.add_format({'bold': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'text_wrap': True,
                                     'fg_color': '#4295f5',
                                     'border': 1})
for col_num, value in enumerate(dfPhaiTraTai3.columns.values):
    worksheet1.write(0, col_num + 1, value, header_format)

for col_num, value in enumerate(dfPhaiTraTaiDuDieuKien.columns.values):
    worksheet2.write(0, col_num + 1, value, header_format)

for col_num, value in enumerate(dfPhaiTraTaiChuaDuDieuKien.columns.values):
    worksheet3.write(0, col_num + 1, value, header_format)

worksheet1.freeze_panes(1, 0)
worksheet2.freeze_panes(1, 0)
worksheet3.freeze_panes(1, 0)

writer.save()